# Galaxy Classification - Random Forest 

In [7]:
# Import required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from astropy import units as u
from astropy import constants as const
from astropy.visualization import quantity_support
from astropy.coordinates import SkyCoord

In [8]:
# Import and adjust matplotlib dpi scaling for high res displays
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

In [9]:
df = pd.read_csv('modified_galaxy_zoo_2_data.csv')
df

,OBJID,ASSEST_ID,RA,DEC,gz2_class,GZ1_MORPHOLOGY,STAR_FORMING,AGN_LINER,PETROR50_R,PETROR90_R,...,REDSHIFT,PETROMAG_MU,PETROMAG_MG,PETROMAG_MR,PETROMAG_MI,PETROMAG_MZ,PETROR50_R_KPC,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN
0,587731186203885750,278415,0.006464,-0.092593,Sb2l,0,0,0,1.552346,4.724314,...,0.078132,-18.451681,-20.106420,-20.834558,-21.150173,-21.371984,2.294145,22,63,6
1,587731187277693072,280232,0.019752,0.781716,SBc2m,0,0,0,4.174540,9.485429,...,0.080365,-19.804232,-21.529858,-22.282389,-22.643412,-22.916601,6.329214,63,34,7
2,588015510343123099,294272,0.029687,0.857908,Sc,0,0,0,1.805935,5.665029,...,0.081550,-18.846533,-20.506390,-21.243180,-21.584341,-21.863384,2.774615,27,55,7
3,587731186203951236,278417,0.032579,-0.040578,Sb?t,0,0,1,4.040482,11.246556,...,0.023671,-18.629040,-19.668825,-20.125076,-20.315216,-20.470337,1.929550,19,77,1
4,587731187277693087,280233,0.042151,0.634290,Ei,0,0,0,1.878149,4.611546,...,0.054114,-17.928638,-19.356077,-20.070858,-20.395535,-20.669110,1.977281,19,78,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239692,587731187277627616,280223,359.967256,0.671613,Ei,4,0,0,2.209229,5.250726,...,0.081304,-18.989813,-20.305210,-20.978706,-21.284754,-21.504791,3.384962,33,60,7
239693,587731186740756649,279393,359.969490,0.333354,Sc2m,4,0,0,3.130419,6.483614,...,0.080469,-19.709562,-20.810030,-21.346203,-21.586780,-21.781790,4.751741,47,53,7
239694,588015509806252115,293021,359.973925,0.556181,Sb2t,4,0,0,3.303175,9.889430,...,0.080549,-20.075562,-21.657505,-22.350897,-22.666334,-22.908318,5.018529,50,32,7
239695,587731187277627630,280224,359.984922,0.675102,Ei(o),0,0,0,1.966789,6.132270,...,0.084803,-19.253954,-21.126196,-21.886639,-22.268350,-22.547771,3.130471,31,42,7


In [ ]:
ra = df['RA']
dec = df['DEC']
coord = SkyCoord(ra, dec, unit=(u.degree, u.degree), frame = 'icrs')

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fid.add_subplot(111, projection='mollweide')
ax.scatter(coord.ra.radian, coord.dec.radian, marker = '*', s = 5, color = 'red')
ax.set_xticklabels('14h', '16h', '18h', '20h', '22h', '0h', '2h', '4h', '6h', '8h', '10h')
ax.grid(True)
ax.set_xlabel('Declination (DEC)')
ax.set_ylabel('Right Ascension (RA)')
plt.title('Distribution of Galaxy Zoo 2 Galaxies on the Sky')

In [10]:
df.gz2_class.unique()

array(['Sb2l', 'SBc2m', 'Sc', 'Sb?t', 'Ei', 'Sc2t', 'Sb1t', 'Sc?t',
       'SBb2m(r)', 'Er', 'Sb', 'Sb?m', 'SBb3m', 'SBc3t', 'SBc4l(i)',
       'Ser', 'SBd?t', 'Sc(m)', 'Sb?t(r)', 'Sc?m', 'SBc2t', 'SBc?m', 'Ec',
       'Sb2m', 'Sc3t', 'Ei(o)', 'SBb2l', 'SBb?t', 'SBc?t', 'Sc1t',
       'Sb(o)', 'Sc3m', 'SBc2l', 'SBb?m', 'Sen', 'Sb2t', 'Sc1m(r)',
       'SBb2t', 'SBb?m(r)', 'Sc+t', 'Sb(m)', 'SBb2m', 'Sb2m(r)',
       'Sc1t(d)', 'Sb1l(i)', 'SBb2t(r)', 'SBc4t', 'SBc3m', 'Sc(r)',
       'Sb1l(m)', 'SBc2t(r)', 'Sb3t', 'Sc?l', 'Sc2m', 'SBc2m(o)', 'SBd1t',
       'SBc?m(d)', 'Sb?l', 'SBb?t(r)', 'Sa', 'Ec(i)', 'Ser(r)', 'Sa+t(o)',
       'Sd2l(i)', 'Sc?m(i)', 'Sc2l', 'SBb+t', 'SBb4m', 'SBc3t(r)', 'SBb',
       'Sa(o)', 'SBc(m)', 'SBc2m(r)', 'Sb(r)', 'Ei(m)', 'Ei(u)',
       'Sc1m(m)', 'Er(r)', 'Sb?l(d)', 'Sb?m(r)', 'SBc?m(i)', 'Sb2t(r)',
       'Sb1m(d)', 'Er(m)', 'SBc1m(i)', 'Sb?m(i)', 'SBc', 'Sc2t(r)',
       'SBb3t', 'Sc4t', 'Sc2t(m)', 'SBb2m(d)', 'Sc?t(m)', 'Sb?m(m)',
       'Sb3l(m)', 'SBb

In [14]:
df['new_col'] = df['gz2_class'].astype(str).str[0:3]
df['new_col'] = df['new_col'].str.replace('\d+', '')
df['new_col'] = df['new_col'].str.replace('\W', '')
df['new_col'] = df['new_col'].str.replace('Seb', 'Se')
df['new_col'] = df['new_col'].str.replace('Sen', 'Se')
df['new_col'] = df['new_col'].str.replace('Ser', 'Se')
df

<ipython-input-14-fa5cb024f9f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_col'] = df['new_col'].str.replace('\d+', '')
<ipython-input-14-fa5cb024f9f7>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_col'] = df['new_col'].str.replace('\W', '')


,OBJID,ASSEST_ID,RA,DEC,gz2_class,GZ1_MORPHOLOGY,STAR_FORMING,AGN_LINER,PETROR50_R,PETROR90_R,...,PETROMAG_MU,PETROMAG_MG,PETROMAG_MR,PETROMAG_MI,PETROMAG_MZ,PETROR50_R_KPC,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN,new_col
0,587731186203885750,278415,0.006464,-0.092593,Sb2l,0,0,0,1.552346,4.724314,...,-18.451681,-20.106420,-20.834558,-21.150173,-21.371984,2.294145,22,63,6,Sb
1,587731187277693072,280232,0.019752,0.781716,SBc2m,0,0,0,4.174540,9.485429,...,-19.804232,-21.529858,-22.282389,-22.643412,-22.916601,6.329214,63,34,7,SBc
2,588015510343123099,294272,0.029687,0.857908,Sc,0,0,0,1.805935,5.665029,...,-18.846533,-20.506390,-21.243180,-21.584341,-21.863384,2.774615,27,55,7,Sc
3,587731186203951236,278417,0.032579,-0.040578,Sb?t,0,0,1,4.040482,11.246556,...,-18.629040,-19.668825,-20.125076,-20.315216,-20.470337,1.929550,19,77,1,Sb
4,587731187277693087,280233,0.042151,0.634290,Ei,0,0,0,1.878149,4.611546,...,-17.928638,-19.356077,-20.070858,-20.395535,-20.669110,1.977281,19,78,4,Ei
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239692,587731187277627616,280223,359.967256,0.671613,Ei,4,0,0,2.209229,5.250726,...,-18.989813,-20.305210,-20.978706,-21.284754,-21.504791,3.384962,33,60,7,Ei
239693,587731186740756649,279393,359.969490,0.333354,Sc2m,4,0,0,3.130419,6.483614,...,-19.709562,-20.810030,-21.346203,-21.586780,-21.781790,4.751741,47,53,7,Sc
239694,588015509806252115,293021,359.973925,0.556181,Sb2t,4,0,0,3.303175,9.889430,...,-20.075562,-21.657505,-22.350897,-22.666334,-22.908318,5.018529,50,32,7,Sb
239695,587731187277627630,280224,359.984922,0.675102,Ei(o),0,0,0,1.966789,6.132270,...,-19.253954,-21.126196,-21.886639,-22.268350,-22.547771,3.130471,31,42,7,Ei


In [15]:
df.new_col.unique()

array(['Sb', 'SBc', 'Sc', 'Ei', 'SBb', 'Er', 'Se', 'SBd', 'Ec', 'Sa',
       'Sd', 'A', 'SBa'], dtype=object)

In [16]:
X_tr, X_te, y_tr, y_te = train_test_split(df[['PETROR50_R','PETROR90_R','PETROMAG_U','PETROMAG_G',
                                              'PETROMAG_R','PETROMAG_I','PETROMAG_Z','PSFMAG_R','FIBERMAG_R','DEVMAG_R',
                                              'EXPMAG_R','FRACDEV_R','MU50_R','CMODELMAG_R','REDSHIFT','PETROMAG_MU',
                                              'PETROMAG_MG','PETROMAG_MR','PETROMAG_MI','PETROMAG_MZ','PETROR50_R_KPC']], 
                                              df['GZ1_MORPHOLOGY'], test_size = 0.2, random_state = 42)

In [17]:
rf = RandomForestClassifier(n_estimators = 20, max_depth = 10, oob_score = True)
#n_estimators = 20, max_depth = 10, max_features didnt matter
rf.fit(X_tr, y_tr)
y_pr = rf.predict(X_te)
err = np.mean(y_pr != y_te)
err, 1 - rf.oob_score_

(0.2841677096370463, 0.2886935027143729)

In [19]:
rf.feature_names = list(X_tr.columns.values)
print(rf.feature_names)

['STAR_FORMING', 'AGN_LINER', 'PETROR50_R', 'PETROR90_R', 'PETROMAG_U', 'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I', 'PETROMAG_Z', 'PSFMAG_R', 'FIBERMAG_R', 'DEVMAG_R', 'EXPMAG_R', 'FRACDEV_R', 'MU50_R', 'CMODELMAG_R', 'REDSHIFT', 'PETROMAG_MU', 'PETROMAG_MG', 'PETROMAG_MR', 'PETROMAG_MI', 'PETROMAG_MZ', 'PETROR50_R_KPC']


In [21]:
rf_df = pd.DataFrame(columns=['Features', 'Importance'])
rf_df['Features'] = rf.feature_names
rf_df['Importance'] = rf.feature_importances_
rf_df = rf_df.sort_values(by = 'Importance', ascending = False, ignore_index = True)
rf_df

,Features,Importance
0,FRACDEV_R,0.075059
1,PETROR50_R,0.068127
2,MU50_R,0.059884
3,PETROR90_R,0.059501
4,PETROMAG_MZ,0.059325
5,PSFMAG_R,0.058200
6,PETROMAG_MI,0.051888
7,PETROMAG_U,0.049794
8,PETROMAG_MR,0.046829
9,PETROR50_R_KPC,0.044968


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.barh(rf_df.Features[0:30], rf_df.Importance[0:30])
ax.set_xlabel('Importance', fontsize = 14)
ax.set_ylabel('Feature', fontsize = 14)
ax.invert_yaxis()
plt.title("Feature Importances in Random Forest", fontsize = 14)
plt.show()

In [ ]:
'STAR_FORMING','AGN_LINER','PETROR50_R','PETROR90_R','PETROMAG_U','PETROMAG_G',
                                                        'PETROMAG_R','PETROMAG_I','PETROMAG_Z','PSFMAG_R','FIBERMAG_R','DEVMAG_R',
                                                        'EXPMAG_R','FRACDEV_R','MU50_R','CMODELMAG_R','REDSHIFT','PETROMAG_MU',
                                                        'PETROMAG_MG','PETROMAG_MR','PETROMAG_MI','PETROMAG_MZ','PETROR50_R_KPC'

### Grid Search

In [ ]:
rf.get_params()

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_tr, y_tr)

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, test_features, test_labels)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))


In [ ]:
# 10-Fold Cross validation
print np.mean(cross_val_score(clf, X_train, y_train, cv=10))